In [2]:
import pandas
from flask import Flask, url_for
from flask import request
from flask import render_template
from ast import literal_eval
import json

In [3]:
# iniciamos archivos csv (absolute path)
df_recommendations = pandas.read_csv('/Users/andrescarvallo/Desktop/RecSys-Project-2017/datasets_recsys/recommendation_results.csv',  delimiter=',')

# convertimos strings a listas
df_recommendations['recommendation'] = df_recommendations['recommendation'].apply(lambda x: literal_eval(x))

# abrimos el df de stories para obtener mas informacion (absolute path)
df_stories = pandas.read_csv('/Users/andrescarvallo/Desktop/RecSys-Project-2017/datasets_recsys/ff_books_dataset.csv', sep='|', engine='python')

# eliminamos filas con datos malos (fechas, etc...)
df_stories['story_id'] = pandas.to_numeric(df_stories['story_id'], errors='coerce')
df_stories = df_stories.dropna(subset=['story_id'])

# convertimos a int la fila de story_id
id_story = pandas.Series([int(float(x)) for x in list(df_stories['story_id'])])

# creamos nueva columna con ints story id
df_stories['story_id2'] = id_story.values

# proyectamos los datos que nos sirven en definitiva 
df_stories = df_stories[['story_id2', 'story_title', 'author', 'franchise', 'story_summary']]

# cambiamos nombres de las columnas 
df_stories.columns = ['story_id', 'story_title', 'author', 'franchise', 'story_summary']

df_favorites = pandas.read_csv('/Users/andrescarvallo/Desktop/RecSys-Project-2017/datasets_recsys/favorite_stories_books_sample.csv', delimiter=';')

df_favorites.head()

,user_id,story_id,franchise
0,102390,7295117,Harry Potter
1,102390,10629488,Harry Potter
2,102390,10089646,Harry Potter
3,102390,8945573,Harry Potter
4,102390,8037808,Harry Potter


In [ ]:
df_favorites

,user_id,story_id,franchise
0,102390,7295117,Harry Potter
1,102390,10629488,Harry Potter
2,102390,10089646,Harry Potter
3,102390,8945573,Harry Potter
4,102390,8037808,Harry Potter
5,102390,11031294,Harry Potter
6,102390,6512582,Harry Potter
7,102390,10808355,Harry Potter
8,102390,11330784,Harry Potter
9,102390,4377774,Harry Potter


In [ ]:
# funciona con el notebook FF_NOTEBOOK_RECSYS

app = Flask(__name__)

@app.route("/")
def index_page():
    return render_template('index.html')

@app.route('/als')
def my_form():
    return render_template("my-form.html")


@app.route('/', methods=['GET','POST'])
def my_form_post():
    text = int(request.form['text'])
    final_result = ''

    if 'my-form' in request.form:
        rec_stories = list(df_recommendations.loc[df_recommendations['user'] == text]['recommendation'])[0]
        
        result = df_stories[df_stories['story_id'].isin(rec_stories)]
        
        url = pandas.Series(['http://www.fanfiction.net/s/{0}'.format(str(x))  for x in list(result['story_id'])])
        
        # agrega url de la historia recomendada 
        result['url'] = url.values
        
        summary = [x.split(' ') for x in list(result['story_summary'])]
        
        words = []
        for x in summary:
            words.extend(x)
        
        # palabras para el diagrama 
        words = list(set(words))
        
        # favorite stories user 
        fav_stories = df_favorites.loc[df_favorites['user_id'] == text]
        fav_stories = fav_stories.merge(df_stories, on='story_id')
        fav_stories = fav_stories[['user_id', 'story_id', 'franchise_x', 'story_title', 'author', 'story_summary']]
        fav_stories.columns = ['user_id', 'story_id', 'franchise', 'story_title', 'author', 'story_summary'] 
        
        url_favs = pandas.Series(['http://www.fanfiction.net/s/{0}'.format(str(x)) for x in list(fav_stories['story_id'])])
        
        fav_stories['url'] = url_favs.values
         
        fav_stories = fav_stories.to_html()
        
        final_result = result.to_html()
        
    return render_template('view.html',tables=[final_result, fav_stories],titles = ['recommendation'], labels=[text, words, 'recommended', 'favorite'])

if __name__ == '__main__':
    app.run(port=6030)

 * Running on http://127.0.0.1:6030/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Nov/2017 14:32:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2017 14:32:16] "GET /static/fanfiction.jpg HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2017 14:32:16] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [13/Nov/2017 14:32:18] "GET /als HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2017 14:32:18] "GET /static/8139.jpg HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2017 14:32:18] "GET /static/8334.jpg HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2017 14:32:18] "GET /static/17420.jpg HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2017 14:32:18] "GET /static/67319.jpg HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2017 14:32:18] "GET /static/1277221.jpg HTTP/1.1" 200 -
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-d